In [1]:

from translate import *
# process_file("")
import glob
import os
import pandas as pd
# file_names = glob.glob("../C27045761/*.csv")
# file_name = file_names[1]


# Process Original File for better thai revision & english translation
# process_file(file_name, api_key = os.environ['ANTHROPIC_API_KEY'])


# Postprocess file for desired answer & question
file_name = "02_llm.csv"
post_proc_llm(file_name)

 12%|█▎        | 1/8 [00:02<00:17,  2.48s/it]

Query Question
Found related question.... 


 25%|██▌       | 2/8 [00:04<00:14,  2.40s/it]

Query Question
Found related question.... 


 38%|███▊      | 3/8 [00:07<00:12,  2.42s/it]

...

Query Question
Found related question.... 


 88%|████████▊ | 7/8 [00:17<00:02,  2.54s/it]

Query Question
Found related question.... 


100%|██████████| 8/8 [00:20<00:00,  2.54s/it]

Query Question
Found related question.... 
